# Synthetic sEMG and ventilator data maker
IMPORTANT: this notebook requires neurokit2 in the environment

In [ ]:
# Standard code libraries
import sys

import scipy
from scipy import signal
from scipy import interpolate as interp
from scipy.optimize import curve_fit

import numpy as np
import numpy.matlib
import pandas as pd

import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime

import neurokit2 as nk
import random

%matplotlib widget

In [ ]:
# Custom code libraries from the ReSurfEMG repository

# TODO: Eliminate sys.path.insert(0, '..')
sys.path.insert(0, '..')
from resurfemg.config.config import simulate_emg_with_occlusions
from resurfemg.config.config import simulate_ventilator_with_occlusions

import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt
import resurfemg.postprocessing.features as feat



In [ ]:
# Recording parameters
t_rec = 7*60    # Recording duration (s)
fs_vent = 100   # Sampling rate of ventilator (Hz)
fs_emg = 2048   # Sampling rate of EMG amplifier (Hz)

# Ventilator parameters
peep = 5        # Positive end-expiratory pressure (cmH2O)
dp = 5          # Driving pressure above PEEP (cmH2O)  

# Patient respiratory variables
ie_ratio = 1/2  # Ratio between inspiratory and expiratory time
rr = 22         # Respiratory rate (/min)
p_mus_max = 5   # Maximal respiratory muscle pressure (cmH2O)
c = .050        # Respiratory system compliance (L/cmH2O)
r = 5           # Respiratory system resistance (cmH2O/L/s)
p_ip = -5       # Static interpleural pressure (cmH2O)

tau_mus_up = 0.3    # Muscle contraction time constant
tau_mus_down = 0.3  # Muscle release time constant

# Cardiac parameters
hr_min = 60     # Minimal heart rate (bpm)
hr_max = 100    # Maximal heart rate (bpm)

# Occlussion manoeuvre (Pocc) settings
t_occs = np.array([t_rec-45, t_rec-30, t_rec-15])
t_occs = np.floor(t_occs*rr/60)*60/rr   # Sync Poccs with respiratory pattern
for i, t_occ in enumerate(t_occs):
    if t_rec < (t_occ + 60/rr):
        print('t=' + str(t_occ) + ': t_occ should be at least a full '
              + 'respiratory cycle from t_end')

In [ ]:
# Generate ECG data
hr = random.randint(hr_min, hr_max)
part_ecg = nk.ecg_simulate(duration=int(t_rec/1.5), 
                           sampling_rate=int(fs_emg*1.5), 
                           heart_rate=hr) 

print('The randomised heart rate is set at: ', hr, 'bpm')

In [ ]:
# Generate sEMGdi data
emg_di_part = simulate_emg_with_occlusions(
    occs_times_vals=t_occs,
    t_start=0,
    t_end=t_rec,
    emg_fs=fs_emg,
    rr=rr,         
    ie_ratio=ie_ratio,
    tau_mus_up=0.3,
    tau_mus_down=0.3,
    emg_amp=5,     # Approximate EMG-RMS amplitude (uV)
    drift_amp=100, # Approximate drift RMS amplitude (uV)
    noise_amp=2    # Approximate baseline noise RMS amplitude (uV)
)

In [ ]:
# Combine ECG and sEMGdi data
y_emg_remix = np.zeros((2, emg_di_part.shape[0]))
y_emg_remix[0] = 200 * part_ecg + 0.2 * emg_di_part     # The ECG channel
y_emg_remix[1] = 200 * part_ecg + 1.0 * emg_di_part     # The sEMGdi channel

In [ ]:
y_vent = simulate_ventilator_with_occlusions(
    occs_times_vals=t_occs,     # Timing of occlusions (s)
    t_start=0,
    t_end=t_rec,
    fs_vent=fs_vent,            # hertz
    rr=rr,                      # respiratory rate /min
    ie_ratio=ie_ratio,          # Ratio between inspiratory and expiratory time
    p_mus_max=p_mus_max,        # Maximal respiratory muscle pressure (cmH2O)
    tau_mus_up=tau_mus_up,      # Muscle contraction time constant
    tau_mus_down=tau_mus_down,  # Muscle release time constant
    c=c,                        # Respiratory system compliance (L/cmH2O)
    r=r,                        # Respiratory system resistance (cmH2O/L/s)
    peep=peep,                  # Positive end-expiratory pressure (cmH2O)
    dp=dp                       # Driving pressure above PEEP (cmH2O)
)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
# data_emg = Poly5Reader(emg_file_chosen)
# data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = y_emg_remix
emg_fs = fs_emg
data_vent_samples = y_vent
vent_fs = fs_vent

# Define the time series of the EMG and ventilator recordings
y_emg = y_emg_remix
y_vent = y_vent

# Define the time axes
t_emg = [i/emg_fs for i in range(len(y_emg[0, :]))]
t_vent = [i/vent_fs for i in range(len(y_vent[0, :]))]

# Default settings for window of interest including the end-expiratory occlusion
# manoeuvres (Pocc)
t_start_default = t_vent[-1]-60
t_end_default = t_vent[-1]-5

In [ ]:
# Plot the raw data
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)

axis[0, 0].grid(True)
axis[0, 0].plot(t_emg, y_emg[0])
axis[0, 0].set(title='sEMG leads')
axis[0, 0].set_ylabel('ECG (uV)')
axis[1, 0].plot(t_emg, y_emg[1])
axis[1, 0].set_ylabel('sEMGdi (uV)')
axis[1, 0].set_xlabel('t (s)')

axis[0, 1].set(title='Ventilator data')
axis[0, 1].grid(True)
axis[0, 1].plot(t_vent, y_vent[0])
axis[0, 1].set_ylabel('Paw (cmH2O)')
axis[1, 1].plot(t_vent, y_vent[1])
axis[1, 1].set_ylabel('F (L/min)')
axis[2, 1].plot(t_vent, y_vent[2])
axis[2, 1].set_ylabel('V (mL)')
axis[2, 1].set_xlabel('t (s)')

## 3. Select the time window of interest

Enter the start and end time of the window in which the occlusions occur. Make sure to take some margin around the peaks in the order of a couple of seconds to allow for the filtering algorithms to work properly. On the other hand, if the window is to broad, the algorithms will also take longer to evaluate.

In [ ]:
# Window selection

start_widget = widgets.BoundedFloatText(
    value=np.floor(t_start_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='Start:',
    disabled=False
)
end_widget = widgets.BoundedFloatText(
    value=np.floor(t_end_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='End:',
    disabled=False
)
widgets.VBox([start_widget, end_widget])

In [ ]:
# Plot the selected window if valid start and end times are entered
start = start_widget.value  
end = end_widget.value  
if ((0 > start) | (t_vent[-1] < start) | (0 > end) | (t_vent[-1] < end) 
    | (start >= end)):    
    print('Please make sure that 1) the start and end times are within the'  
          +' limits of the recording and 2) that the start time is before the' 
          + ' end time.')
else:
    # Plot the selected window.
    start_idx = int(float(start)* emg_fs)
    end_s = min([int(float(end)*emg_fs), len(y_emg[0,:])-1])
    start_vent_s = int(float(start)* vent_fs)
    end_vent_s = min(
        [int(float(end)* vent_fs), len(y_vent[0,:])-1]
    )

    fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(12, 6), sharex=True)
    axis[0, 0].set(title='sEMG leads')
    axis[0, 0].grid(True)
    axis[0, 0].plot(t_emg[int(start_idx):int(end_s)], 
                    y_emg[0][int(start_idx):int(end_s)])
    axis[0, 0].set_ylabel('ECG (uV)')
    axis[1, 0].plot(t_emg[int(start_idx):int(end_s)], 
                    y_emg[1][int(start_idx):int(end_s)])
    axis[1, 0].set_ylabel('sEMGdi (uV)')
    axis[1, 0].set_xlabel('t (s)')
    
    axis[0, 1].set(title='Ventilator data')
    axis[0, 1].grid(True)
    axis[0, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[0][int(start_vent_s):int(end_vent_s)]
    )
    axis[0, 1].set_ylabel('Paw (cmH2O)')
    axis[1, 1].grid(True)
    axis[1, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[1][int(start_vent_s):int(end_vent_s)]
    )
    axis[1, 1].set_ylabel('F (L/min)')
    axis[2, 1].grid(True)
    axis[2, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[2][int(start_vent_s):int(end_vent_s)]
    )
    axis[2, 1].set_ylabel('V (mL)')
    axis[2, 1].set_xlabel('t (s)')